# Exploring the Impact of Home Team Advantage in Football

## Description
Analyze whether playing at home significantly increases a football team's chance of winning.

## Table of Contents
1. [Executive Summary](#executive-summary)
2. [Introduction & Background](#introduction--background)
3. [Imports](#imports)
4. [Data & API](#data-to-explore)
5. [Analysis](#analysis)
6. [Conclusion](#conclusion)

## Executive Summary

## Introduction & Background

Home field advantage is a long-discussed phenomenon in sports. This analysis investigates whether such an advantage exists in football and, if so, how strong it is across different leagues and teams. This is often a key indictor used in football match predictions and betting algorithms. We will use both frequentist and bayesian methods to analyze the data to answer these questions.

## Imports

In [1]:
# Allows custom modules to be auto reloaded
%load_ext autoreload
%autoreload 2

In [2]:
import soccerdata as sd
import pandas as pd
import numpy as np
from football_analytics.paths import RAW_DATA_DIR

import logging
# Optional: suppresses noisy logs
logging.getLogger().setLevel(logging.ERROR)

[05/05/25 19:00:28] INFO     No custom team name replacements found. You can configure these in       _config.py:91
                             C:\Users\DELL\soccerdata\config\teamname_replacements.json.                           

                    INFO     No custom league dict found. You can configure additional leagues in    _config.py:197
                             C:\Users\DELL\soccerdata\config\league_dict.json.                                     

## Data & API

To gather the data, we will use the module soccerdata: a web scraper that provides accurate data from a variety of different websites on football. Specifically we use soccerdata to collect match history from the website [football-data](https://www.football-data.co.uk/data.php). To provide a large amount of data for analysis, we will look at the five biggest european leagues over a span of 20 years.

In [11]:
TIME_PERIOD = 20 # How far back are we looking in years

match_history = sd.MatchHistory(
    leagues=[
        'ENG-Premier League', 
        'ESP-La Liga', 
        'FRA-Ligue 1', 
        'GER-Bundesliga', 
        'ITA-Serie A'
    ],
    seasons=range(2025 - TIME_PERIOD, 2025),
    no_cache=False,
    no_store=False,
    data_dir=RAW_DATA_DIR / "home advantage"
)

games = match_history.read_games()

The dataframe contains matches indexed by the league, season, and the specific game. It records data such as the home team, away team, home goals (FTHG), away goals (FTAG), betting measurements, and other match details.

In [12]:
games.sample(5)

date  \
league             season game                                                 
ITA-Serie A        0607   2007-02-24 Atalanta-Palermo    2007-02-24 12:00:00   
GER-Bundesliga     2425   2025-03-15 Mainz-Freiburg      2025-03-15 14:30:00   
ENG-Premier League 2425   2024-12-22 Everton-Chelsea     2024-12-22 14:00:00   
ESP-La Liga        1112   2012-02-05 Ath Madrid-Valencia 2012-02-05 12:00:00   
                   1617   2017-05-21 Barcelona-Eibar     2017-05-21 12:00:00   

                                                           home_team  \
league             season game                                         
ITA-Serie A        0607   2007-02-24 Atalanta-Palermo       Atalanta   
GER-Bundesliga     2425   2025-03-15 Mainz-Freiburg            Mainz   
ENG-Premier League 2425   2024-12-22 Everton-Chelsea         Everton   
ESP-La Liga        1112   2012-02-05 Ath Madrid-Valencia  Ath Madrid   
                   1617   2017-05-21 Barcelona-Eibar       Barcelona   

                                                         away_team  FTHG  \
league             season game                                             
ITA-Serie A        0607   2007-02-24 Atalanta-Palermo      Palermo   1.0   
GER-Bundesliga     2425   2025-03-15 Mainz-Freiburg       Freiburg   2.0   
ENG-Premier League 2425   2024-12-22 Everton-Chelsea       Chelsea   0.0   
ESP-La Liga        1112   2012-02-05 Ath Madrid-Valencia  Valencia   0.0   
                   1617   2017-05-21 Barcelona-Eibar         Eibar   4.0   

                                                          FTAG FTR  HTHG  \
league             season game                                             
ITA-Serie A        0607   2007-02-24 Atalanta-Palermo      1.0   D   1.0   
GER-Bundesliga     2425   2025-03-15 Mainz-Freiburg        2.0   D   1.0   
ENG-Premier League 2425   2024-12-22 Everton-Chelsea       0.0   D   0.0   
ESP-La Liga        1112   2012-02-05 Ath Madrid-Valencia   0.0   D   0.0   
                   1617   2017-05-21 Barcelona-Eibar       2.0   H   0.0   

                                                          HTAG HTR  \
league             season game                                       
ITA-Serie A        0607   2007-02-24 Atalanta-Palermo      0.0   H   
GER-Bundesliga     2425   2025-03-15 Mainz-Freiburg        0.0   H   
ENG-Premier League 2425   2024-12-22 Everton-Chelsea       0.0   D   
ESP-La Liga        1112   2012-02-05 Ath Madrid-Valencia   0.0   D   
                   1617   2017-05-21 Barcelona-Eibar       1.0   A   

                                                             referee  ...  \
league             season game                                        ...   
ITA-Serie A        0607   2007-02-24 Atalanta-Palermo     R. Rosetti  ...   
GER-Bundesliga     2425   2025-03-15 Mainz-Freiburg              NaN  ...   
ENG-Premier League 2425   2024-12-22 Everton-Chelsea      C Kavanagh  ...   
ESP-La Liga        1112   2012-02-05 Ath Madrid-Valencia         NaN  ...   
                   1617   2017-05-21 Barcelona-Eibar             NaN  ...   

                                                          1XBCH  1XBCD  1XBCA  \
league             season game                                                  
ITA-Serie A        0607   2007-02-24 Atalanta-Palermo       NaN    NaN    NaN   
GER-Bundesliga     2425   2025-03-15 Mainz-Freiburg        1.83   3.61   4.83   
ENG-Premier League 2425   2024-12-22 Everton-Chelsea       4.90   4.24   1.70   
ESP-La Liga        1112   2012-02-05 Ath Madrid-Valencia    NaN    NaN    NaN   
                   1617   2017-05-21 Barcelona-Eibar        NaN    NaN    NaN   

                                                          BFECH  BFECD  BFECA  \
league             season game                                                  
ITA-Serie A        0607   2007-02-24 Atalanta-Palermo       NaN    NaN    NaN   
GER-Bundesliga     2425   2025-03-15 Mainz-Freiburg        1.88    3.7   5.00   
ENG-Premier Leagu

We only care about whether a team was home or away and whether it won or not. So we can drop all irrelevant columns and leave only the home and away teams and goals

In [13]:
games = games[["date", "home_team", "away_team", "FTHG", "FTAG"]]
games.sample(5)

date  \
league             season game                                                         
FRA-Ligue 1        0708   2007-12-15 Paris SG-Toulouse           2007-12-15 12:00:00   
GER-Bundesliga     0506   2006-05-13 Hamburg-Werder Bremen       2006-05-13 12:00:00   
ITA-Serie A        2021   2021-04-24 Parma-Crotone               2021-04-24 17:00:00   
                   2425   2024-11-03 Napoli-Atalanta             2024-11-03 11:30:00   
ENG-Premier League 2021   2020-12-17 Sheffield United-Man United 2020-12-17 20:00:00   

                                                                         home_team  \
league             season game                                                       
FRA-Ligue 1        0708   2007-12-15 Paris SG-Toulouse                    Paris SG   
GER-Bundesliga     0506   2006-05-13 Hamburg-Werder Bremen                 Hamburg   
ITA-Serie A        2021   2021-04-24 Parma-Crotone                           Parma   
                   2425   2024-11-03 Napoli-Atalanta                        Napoli   
ENG-Premier League 2021   2020-12-17 Sheffield United-Man United  Sheffield United   

                                                                      away_team  \
league             season game                                                    
FRA-Ligue 1        0708   2007-12-15 Paris SG-Toulouse                 Toulouse   
GER-Bundesliga     0506   2006-05-13 Hamburg-Werder Bremen        Werder Bremen   
ITA-Serie A        2021   2021-04-24 Parma-Crotone                      Crotone   
                   2425   2024-11-03 Napoli-Atalanta                   Atalanta   
ENG-Premier League 2021   2020-12-17 Sheffield United-Man United     Man United   

                                                                  FTHG  FTAG  
league             season game                                                
FRA-Ligue 1        0708   2007-12-15 Paris SG-Toulouse             1.0   2.0  
GER-Bundesliga     0506   2006-05-13 Hamburg-Werder Bremen         1.0   2.0  
ITA-Serie A        2021   2021-04-24 Parma-Crotone                 3.0   4.0  
                   2425   2024-11-03 Napoli-Atalanta               0.0   3.0  
ENG-Premier League 2021   2020-12-17 Sheffield United-Man United   2.0   3.0

Now we can check if we need to clean the data by finding missing values or duplicate data.

In [14]:
games.isnull().any() # Returns true if there are any missing values

date         False
home_team    False
away_team    False
FTHG         False
FTAG         False
dtype: bool

In [15]:
duplicates = games[games.duplicated(keep=False)]
duplicates.sort_values(by=['date'])

date  \
league      season game                                                
FRA-Ligue 1 2021   2020-08-21 Bordeaux-Nantes    2020-08-21 18:00:00   
                   2020-08-21 Bordeaux-Nantes    2020-08-21 18:00:00   
                   2020-08-22 Dijon-Angers       2020-08-22 16:00:00   
                   2020-08-22 Dijon-Angers       2020-08-22 16:00:00   
                   2020-08-22 Lille-Rennes       2020-08-22 20:00:00   
...                                                              ...   
                   2021-05-23 Rennes-Nimes       2021-05-23 20:00:00   
                   2021-05-23 St Etienne-Dijon   2021-05-23 20:00:00   
                   2021-05-23 St Etienne-Dijon   2021-05-23 20:00:00   
                   2021-05-23 Lyon-Nice          2021-05-23 20:00:00   
                   2021-05-23 Nantes-Montpellier 2021-05-23 20:00:00   

                                                   home_team    away_team  \
league      season game                                                     
FRA-Ligue 1 2021   2020-08-21 Bordeaux-Nantes       Bordeaux       Nantes   
                   2020-08-21 Bordeaux-Nantes       Bordeaux       Nantes   
                   2020-08-22 Dijon-Angers             Dijon       Angers   
                   2020-08-22 Dijon-Angers             Dijon       Angers   
                   2020-08-22 Lille-Rennes             Lille       Rennes   
...                                                      ...          ...   
                   2021-05-23 Rennes-Nimes            Rennes        Nimes   
                   2021-05-23 St Etienne-Dijon    St Etienne        Dijon   
                   2021-05-23 St Etienne-Dijon    St Etienne        Dijon   
                   2021-05-23 Lyon-Nice                 Lyon         Nice   
                   2021-05-23 Nantes-Montpellier      Nantes  Montpellier   

                                                  FTHG  FTAG  
league      season game                                       
FRA-Ligue 1 2021   2020-08-21 Bordeaux-Nantes      0.0   0.0  
                   2020-08-21 Bordeaux-Nantes      0.0   0.0  
                   2020-08-22 Dijon-Angers         0.0   1.0  
                   2020-08-22 Dijon-Angers         0.0   1.0  
                   2020-08-22 Lille-Rennes         1.0   1.0  
...                                                ...   ...  
                   2021-05-23 Rennes-Nimes         2.0   0.0  
                   2021-05-23 St Etienne-Dijon     0.0   1.0  
                   2021-05-23 St Etienne-Dijon     0.0   1.0  
                   2021-05-23 Lyon-Nice            2.0   3.0  
                   2021-05-23 Nantes-Montpellier   1.0   2.0  

[3652 rows x 5 columns]

In [16]:
prev = len(games)
games = games.drop_duplicates()
print(f"Removed {prev - len(games)} duplicate values")

Removed 1826 duplicate values
